In [1]:
# To reload external files automatically (ex: utils)
%load_ext autoreload
%autoreload 2

import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle # For rectangles
import proplot as plot # New plot library (https://proplot.readthedocs.io/en/latest/)
from scipy import stats # For linregress (trends)
import xesmf as xe # For regridding (https://xesmf.readthedocs.io/en/latest/)

import sys
sys.path.insert(1, '/home/mlalande/notebooks') # to include my util file in previous directory
import utils as u # my personal functions
u.check_python_version()
u.check_virtual_memory()

3.8.2 | packaged by conda-forge | (default, Feb 28 2020, 17:15:22) 
[GCC 7.3.0]
Virtual memory usage - total: 252 GB / available: 163 GB / percent used: 35.4 %


In [2]:
path = '/data/mlalande/MODIS/MOD10CM/hdf/'

In [3]:
files = !ls {path}*hdf

In [10]:
import xml.etree.ElementTree as ET

grid = xe.util.grid_global(d_lon=0.05, d_lat=0.05)

for file in files:
    print(file)
    ds = xr.open_dataset(file)
    
    # Add grid coords
    ds = ds.rename({"XDim:MOD_CMG_Snow_5km": "lon", "YDim:MOD_CMG_Snow_5km": "lat"})
    ds = ds.assign_coords({"lon": grid.lon[0].values, "lat": np.flip(grid.lat[:,0].values)})
    
    # Get date
    tree = ET.parse(file+'.xml')
    root = tree.getroot()

    for RangeBeginningDate in root.iter('RangeBeginningDate'):
        start = np.datetime64(RangeBeginningDate.text)

    for RangeEndingDate in root.iter('RangeEndingDate'):
        end = np.datetime64(RangeEndingDate.text)

    time = (end - start)/2 + start
    ds = ds.assign_coords(time=time).expand_dims('time')
    ds.to_netcdf(path+'../netcdf/MOD10CM_0.05_'+str(time)[:-3]+'.nc')

/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2000061.006.2016063065826.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2000092.006.2016065155122.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2000122.006.2016065173135.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2000153.006.2016068063433.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2000183.006.2016069041258.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2000245.006.2016069160249.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2000275.006.2016069163705.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2000306.006.2016069175525.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2000336.006.2016077194332.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2001001.006.2016092173324.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2001032.006.2016092173150.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2001060.006.2016097191234.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2001091.006.2016096023059.hdf
/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2001121.006.2016096172

FileNotFoundError: [Errno 2] No such file or directory: '/data/mlalande/MODIS/MOD10CM/hdf/MOD10CM.A2016306.006.2016337082701.hdf.xml'